#**Swire Coca-Cola Cart Abandonment Analysis**
### MSBA Capstone  
**Author:** Alphonsinah Ototo  

---

# Table of Contents  
1. Introduction
2. Data Sources & Loading  
3. Exploratory Data Analysis  
   - Customer segmentation  
   - Ordering behavior  
   - Google Analytics usage behavior  
   - Operational cadence & cutoff times  
   - Product catalog insights  
4. Key EDA Insights  
5. Modeling Workflow  
   - Target definition  
   - Leakage handling  
   - Feature engineering  
   - Train/test split (time-based)  
6. Model Training & Evaluation  
   - Logistic regression  
   - ROC, confusion matrix, classification report  
   - Calibration & Brier score  
7. Profit-Based Threshold Optimization  
8. Final Recommendations & Next Steps  


# 1. Introduction

Swire Coca-Cola’s MyCoke360 platform enables B2B customers to browse products, place orders, and manage deliveries.  
However, many sessions involve product interaction but do not result in a completed purchase, leading to significant cart abandonment.

Cart abandonment matters because it affects:

- Lost revenue  
- Increased operational workload from manual follow-ups  
- Inefficient fulfillment planning    

This notebook contains my individual EDA and modeling work for the MSBA Capstone.  
My goal is to:

1. Understand customer behavior across several operational datasets.  
2. Identify key predictors associated with cart abandonment.  
3. Build a leakage-safe model to estimate purchase probability.  
4. Use profit-based thresholding to determine which customers are worth contacting.  

This notebook represents my personal contribution to the group project.




# 2. Data Sources & Loading

The analysis uses multiple operational datasets, including:

- **Customer master**
- **Sales orders**
- **Google Analytics event logs**
- **Operating hours & order frequency**
- **Cutoff times**
- **Product catalog**
- **Visit plan history**



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
data_path = "/content/drive/MyDrive/Swire_CocaCola_Data"
os.listdir(data_path)

In [ ]:
import pandas as pd

customer = pd.read_csv(data_path + "/customer.csv")
sales = pd.read_csv(data_path + "/sales.csv")
google_analytics = pd.read_csv(data_path + "/google_analytics.csv")
operating_hours = pd.read_csv(data_path + "/operating_hours.csv")
cutoff_times= pd.read_csv(data_path + "/cutoff_times.csv")
orders = pd.read_csv(data_path + "/ orders.csv")
materials = pd.read_csv(data_path + "/material.csv")
visit_plan = pd.read_csv(data_path + "/visit_plan.csv")



# 3. Exploratory Data Analysis (EDA)
This section explores customer behavior, ordering patterns, event-level engagement, and operational constraints that may contribute to cart abandonment.



In [ ]:
#Preview
customer.head()

In [ ]:
customer.info()
customer.shape

In [ ]:
# Checking for missing values
customer.isnull().sum()

In [ ]:
missing_pct = (customer['DISTRIBUTION_MODE_DESCRIPTION'].isnull().sum() / len(customer)) * 100
print(f"Missing values in DISTRIBUTION_MODE_DESCRIPTION: {missing_pct:.2f}%")

 The dataset contains 6,334 customers with complete information across most fields.  
Distribution mode has minimal data missing (0.06%)


In [ ]:
#Dropping Null Values since they have a minimal effect on the dataset
customer = customer.dropna(subset=['DISTRIBUTION_MODE_DESCRIPTION'])

In [ ]:
# Duplicates
customer.duplicated().sum()


In [ ]:
# Unique values
print(customer['SALES_OFFICE'].nunique())
print(customer['SALES_OFFICE_DESCRIPTION'].nunique())
print(customer['DISTRIBUTION_MODE_DESCRIPTION'].value_counts())
print(customer['SHIPPING_CONDITIONS_DESCRIPTION'].value_counts())
print(customer['COLD_DRINK_CHANNEL_DESCRIPTION'].value_counts())
print(customer['CUSTOMER_SUB_TRADE_CHANNEL_DESCRIPTION'].value_counts())



- The customer dataset represents 44 unique sales offices, showing extensive geographic coverage across Swire Coca-Cola’s operational regions.
- Distribution is dominated by OFS, Sideload, and Rapid Delivery modes, while shipping is highly standardized, with 95% of customers operating on a 48-hour delivery schedule.
- The primary cold drink channels are Restaurants and Stores
- Further segmentation reveals over 50 customer sub-trade channels, led by Dining, Restaurants, and Local Convenience Stores.

In [ ]:
sales.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

office_counts = (
    customer.groupby("SALES_OFFICE_DESCRIPTION")["CUSTOMER_NUMBER"]
    .nunique()
    .sort_values(ascending=False)
)

plt.figure(figsize=(10,8))
sns.barplot(x=office_counts.values, y=office_counts.index, palette="Reds_r")
plt.title("Number of Unique Customers by Sales Office", fontsize=14)
plt.xlabel("Customer Count")
plt.ylabel("Sales Office")
plt.tight_layout()
plt.show()


In [ ]:
dist_mode_channel = pd.crosstab(
    customer['DISTRIBUTION_MODE_DESCRIPTION'],
    customer['COLD_DRINK_CHANNEL_DESCRIPTION']
)

plt.figure(figsize=(10,6))
sns.heatmap(dist_mode_channel, annot=True, fmt='d', cmap='YlOrRd')
plt.title("Distribution Mode by Cold Drink Channel", fontsize=14)
plt.xlabel("Cold Drink Channel")
plt.ylabel("Distribution Mode")
plt.tight_layout()
plt.show()


- heatmap illustrates that OFSand Sideload are the dominant delivery modes across all cold drink channels, especially for Restaurants and Stores.
- Rapid Delivery is mainly used by Restaurants, Hot Beverage, and Attraction customers. This suggests that most customers are served through standard delivery modes, with only a few channels relying on faster or specialized logistics.

In [ ]:
print(google_analytics.head())

print(google_analytics.info())
print(google_analytics.shape)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# Convert EVENT_DATE and EVENT_TIMESTAMP to datetime
google_analytics['EVENT_DATE'] = pd.to_datetime(google_analytics['EVENT_DATE'])
google_analytics['EVENT_TIMESTAMP'] = pd.to_datetime(google_analytics['EVENT_TIMESTAMP'])

# Check conversion
print(google_analytics[['EVENT_DATE', 'EVENT_TIMESTAMP']].head())


In [ ]:

# Replace google_analytics placeholders with NaN
google_analytics.replace(["(not set)", ""], np.nan, inplace=True)

# Check missing values per column
missing_summary = google_analytics.isnull().sum()
print("\nMissing values per column:")
print(missing_summary)


In [ ]:
# Count duplicates before removing
print("Number of duplicate rows:", google_analytics.duplicated().sum())

# Drop duplicates
google_analytics = google_analytics.drop_duplicates()

# Confirm removal
print("Rows after removing duplicates:", len(google_analytics))


In [ ]:
# Basic info
print("\nDataset shape:", google_analytics.shape)
print("\nData types:")
print(google_analytics.dtypes)

# Quick glance at the first few rows
print("\nHead of dataset:")
print(google_analytics.head())


In [ ]:
top_events = google_analytics['EVENT_NAME'].value_counts().head(10)

plt.figure(figsize=(10,6))
sns.barplot(x=top_events.values, y=top_events.index, palette="viridis")
plt.title("Top 10 Google Analytics Events")
plt.xlabel("Number of Events")
plt.ylabel("Event Name")
plt.show()

- Most user interactions involve browsing such as page views, item list views, and button clicks, while fewer users progressed to transactional actions like add_to_cart and proceed_to_checkout.

In [ ]:
# Device category counts
print("\nDevice categories:")
print(google_analytics['DEVICE_CATEGORY'].value_counts())

# Top mobile brands
print("\nTop 10 mobile brands:")
print(google_analytics['DEVICE_MOBILE_BRAND_NAME'].value_counts().head(10))

# Operating systems
print("\nOperating systems:")
print(google_analytics['DEVICE_OPERATING_SYSTEM'].value_counts())


- Most users access the site from desktops, primarily on Windows and Macintosh, while mobile users mainly on Google/Android and Apple/iOS make up a smaller portion.

In [ ]:
events_per_day = google_analytics.groupby('EVENT_DATE')['EVENT_NAME'].count()

plt.figure(figsize=(12,6))
events_per_day.plot(kind='line', marker='o')
plt.title("Daily Events Trend")
plt.xlabel("Date")
plt.ylabel("Number of Events")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

- The daily events trend shows a steady increase in user activity on the MyCoke360 platform from June 2024 through May 2025
- The upward trajectory suggests growing platform adoption and higher engagement

In [ ]:
print(orders.head())
print(orders.info())
print(orders.shape)

In [ ]:
# Check missing values
print(orders.isnull().sum())

# Summary of duplicates
print(orders.duplicated().sum())

# Count distinct customers and products
print(orders['CUSTOMER_ID'].nunique(), orders['MATERIAL_ID'].nunique())


In [ ]:
#orders distribution
print(orders['ORDER_TYPE'].value_counts(normalize=True) * 100)


- Most orders (61.4%) are placed through Sales Representatives, while MyCoke360 accounts for 27.4%

In [ ]:
orders['ORDER_QUANTITY'].describe()

# Top ordered products
orders.groupby('MATERIAL_ID')['ORDER_QUANTITY'].sum().sort_values(ascending=False).head(10)


In [ ]:

# Convert dates
orders['CREATED_DATE_EST'] = pd.to_datetime(orders['CREATED_DATE_EST'])

# Group by date and order type
order_trend = (
    orders.groupby(['CREATED_DATE_EST', 'ORDER_TYPE'])
    .size()
    .reset_index(name='Order_Count')
)

# Plot
plt.figure(figsize=(12,6))
sns.lineplot(
    data=order_trend,
    x='CREATED_DATE_EST',
    y='Order_Count',
    hue='ORDER_TYPE',
    marker='o'
)
plt.title('Daily Order Volume by Channel (Sales Rep vs Call Center)', fontsize=14)
plt.xlabel('Date')
plt.ylabel('Number of Orders')
plt.legend(title='Order Type')
plt.tight_layout()
plt.show()


- Sales Representatives drive the majority of daily orders, showing strong and consistent weekly peaks.

- MyCoke360 orders are steadily increasing.

In [ ]:
print(operating_hours.head())

In [ ]:
import pandas as pd

# Convert dates
operating_hours['CALLING_ANCHOR_DATE'] = pd.to_datetime(operating_hours['CALLING_ANCHOR_DATE'], errors='coerce')

# Missing values
print("Missing values per column:")
print(operating_hours.isnull().sum())

# Duplicates
print("\nNumber of duplicate rows:")
print(operating_hours.duplicated().sum())
#frequency
print(operating_hours['FREQUENCY'].value_counts())

In [ ]:
operating_hours['FREQUENCY'].value_counts().plot(kind='bar', figsize=(8,4), title='Operating hours Distribution ')


- Most operations occur on a regular schedule, with the majority happening every 4 weeks (2,562 records), followed by weekly operations (2,140 records), and every 2 weeks (1,489 records). Only a small portion, 11 records, follow an every 3 weeks pattern.

In [ ]:
print(operating_hours['DELIVERY_ANCHOR_DAY'].value_counts())

# Crosstab: Frequency vs Day
print(pd.crosstab(operating_hours['DELIVERY_ANCHOR_DAY'], operating_hours['FREQUENCY']))


- Most deliveries occur on weekdays, particularly between Tuesday and Friday, with very few scheduled for weekends

In [ ]:
# Customers scheduled for each date
print(operating_hours.groupby("CALLING_ANCHOR_DATE")['CUSTOMER_NUMBER'].count().plot(figsize=(12,6), title="Customers Scheduled per Date"))

# Weekly distribution
print(operating_hours['CALLING_ANCHOR_DATE'].dt.day_name().value_counts())


- The dataset is heavily concentrated around Monday through Wednesday and Sunday, with minimal or almost no calls on Thursday and Friday.

In [ ]:
# Customers with multiple delivery records
print(operating_hours.groupby("CUSTOMER_NUMBER")['CALLING_ANCHOR_DATE'].count().sort_values(ascending=False).head(10))


In [ ]:
print(cutoff_times.head())
print(cutoff_times.info())
print(cutoff_times.shape)

In [ ]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

# Optional: make plots look nicer
sns.set(style="whitegrid")


In [ ]:
# Show raw values exactly as stored
for val in cutoff_times['CUTOFFTIME__C'].head(20):
    print(repr(val))


In [ ]:

cutoff_times['CUTOFFTIME_CLEAN'] = cutoff_times['CUTOFFTIME__C'].astype(str).str.strip()

# Convert to datetime
cutoff_times['CUTOFFTIME_DT'] = pd.to_datetime(cutoff_times['CUTOFFTIME_CLEAN'], errors='coerce')

# Extract only the time
cutoff_times['CUTOFFTIME_ONLY'] = cutoff_times['CUTOFFTIME_DT'].dt.time

# Check results
print("Successfully parsed times:", cutoff_times['CUTOFFTIME_ONLY'].notnull().sum())
print("Failed parsing:", cutoff_times['CUTOFFTIME_ONLY'].isnull().sum())

# Inspect first few rows
print(cutoff_times[['CUTOFFTIME__C', 'CUTOFFTIME_ONLY']].head(10))


In [ ]:
print(cutoff_times['CUTOFFTIME_ONLY'].value_counts().head(10))


In [ ]:
mode_per_plant = cutoff_times.groupby('PLANT_ID')['CUTOFFTIME_ONLY'].agg(lambda x: x.mode()[0])
print(mode_per_plant)


In [ ]:
# Convert 'SHIPPING_CONDITION_TIME' to numeric hours
cutoff_times['SHIPPING_HOURS'] = cutoff_times['SHIPPING_CONDITION_TIME'].str.replace("hrs","").astype(int)

# Quick check
print(cutoff_times[['SHIPPING_CONDITION_TIME', 'SHIPPING_HOURS']].head())


In [ ]:
cutoff_times['SHIPPING_HOURS'].describe()
cutoff_times['SHIPPING_HOURS'].hist(figsize=(8,5))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.boxplot(x='CUTOFFTIME_ONLY', y='SHIPPING_HOURS', data=cutoff_times)
plt.xticks(rotation=45)
plt.title("Shipping Hours vs Cutoff Times")
plt.show()

- The submission deadlines vary between 10:00 AM and 4:00 PM, with the most frequent cutoff windows occurring around 3:00 PM (36), 3:30 PM (27), and 4:00 PM (26).
- These times suggest a strong operational clustering in the mid-afternoon period.
- The earlier cutoff times are associated with shorter shipping durations (24–30 hours), whereas later cutoff times correspond to longer shipping windows (48–72 hours).
- This pattern suggests that regional variations in cutoff scheduling directly influence fulfillment efficiency, where later submission allowances tend to delay shipment cycles.

In [ ]:
print(materials.head())
print(materials.info())
print(materials.shape)

In [ ]:
print(materials.isnull().sum())
print(materials.duplicated().sum())
print(materials.nunique())

In [ ]:
#Fill missing BEV_CAT_DESC values
materials['BEV_CAT_DESC'] = materials['BEV_CAT_DESC'].fillna('Unknown')


In [ ]:
materials['BEV_CAT_DESC'].value_counts(normalize=True)*100


- The beverage portfolio is dominated by Core Sparkling products (36.2%), followed by Energy Drinks (22.5%) and Sports Drinks (11.1%), together accounting for nearly 70% of all SKUs.

In [ ]:
sns.countplot(y='TRADE_MARK_DESC', data=materials, order=materials['TRADE_MARK_DESC'].value_counts().head(10).index)
plt.title('Top 10 Brands by Product Count')


In [ ]:
pd.crosstab(materials['BEV_CAT_DESC'], materials['PACK_TYPE_DESC'])


In [ ]:
print(visit_plan.head())
print(visit_plan.info())
print(visit_plan.shape)

In [ ]:
print(visit_plan.isnull().sum())
print(visit_plan.duplicated().sum())
print(visit_plan.nunique())


In [ ]:
visit_plan['FREQUENCY'].value_counts().plot(kind='bar', figsize=(8,4), title='Distribution of Visit Frequencies')


- The majority of customers are visited weekly

- Less frequent visits (bi-weekly, monthly, or quarterly) occur for smaller or lower-volume customers

In [ ]:
sns.countplot(y='SALES_OFFICE_DESC', data=visit_plan,
              order=visit_plan['SALES_OFFICE_DESC'].value_counts().head(10).index)
plt.title('Top 10 Sales Offices by Planned Visits')


- Draper, UT handles the highest number of planned visits,over 2.5 million, followed by Tempe, AZ and Denver, showing these are major operational hubs.
- Top offices likely serve the largest customer bases or most active routes,

In [ ]:
pd.crosstab(visit_plan['DISTRIBUTION_MODE'], visit_plan['SHIPPING_CONDITIONS_DESC'])


- Most deliveries use 48-hour shipping, dominated by Sideload (SL), Off-Floor (OF), and Route Delivery (RD) modes, showing these as the main distribution channels.

- 24-hour shipments occur less frequently and are concentrated in Route Delivery and Sideload, indicating faster service for select regions or customers.

# 4. Key EDA Insights

- The EDA revealed strong operational consistency across Swire Coca-Cola’s network, with most customers served weekly through 48-hour shipping windows dominated by Sideload and Off-Floor modes.
- Desktop usage is dominant; mobile activity is less frequent.  
- Distribution modes (OFS, Sideload) strongly influence ordering patterns.  
- Cutoff times cluster around mid-afternoon, creating pressure that may lead to incomplete orders.  
- Sales Representatives handled the majority of orders, while MyCoke360 showed growing digital adoption.
- Google Analytics indicated high browsing engagement but lower checkout conversion, supporting the cart abandonment concern.

 Overall, there is clear behavioral and logistical patterns,providing foundation for predicting at-risk customers and improving order completion on the MyCoke360 platform.

# 5. Modeling

## Target Definition  
The target variable made_a_purchase equals 1 if an order was completed in the defined window.

## Leakage Handling  
To ensure valid predictions, I removed all features that may contain leakage, including:  
- Purchase success indicators  
- Completion timestamps  
- Post-checkout event pages  
- Flags explicitly encoding purchase outcome  

## Feature Engineering  
Key features engineered include:
- Time features (hour, day-of-week, month)  
- Device category indicators  
- Event-level behavioral signals (add_to_cart, remove_from_cart, etc.)  
- Item count extracted from JSON lists  

##  Train/Test Split  
To simulate real deployment, I used a **time-based split**:
- First 75% of events → training  
- Last 25% → holdout testing  


In [ ]:

import numpy as np
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

# Loading modelling dataset
file_path = "/content/drive/MyDrive/Final Output for Modeling .csv"
df = pd.read_csv(file_path)

print("Raw shape:", df.shape)
display(df.head())

df.info()
print("\nMissing values per column:")
print(df.isna().sum())

display(df.describe(include='all').transpose())


In [ ]:

import ast

#Normalize placeholder strings to NaN
PLACEHOLDERS = {
    "not available given order type",
    "not available",
    "NA", "N/A", "None", "", "null", "NULL"
}

df_clean = df.copy()

for col in df_clean.columns:
    if df_clean[col].dtype == "object":
        df_clean[col] = df_clean[col].replace(list(PLACEHOLDERS), np.nan)

# Parse datetime columns if present
def to_dt(x):
    return pd.to_datetime(x, errors="coerce")

dt_cols = ["transaction_date_df2", "event_date_df1", "EVENT_TIMESTAMP"]
for c in dt_cols:
    if c in df_clean.columns:
        df_clean[c] = to_dt(df_clean[c])

# Coerce numeric candidates
def to_num(s):
    return pd.to_numeric(s, errors="coerce")

num_candidates = ["ORDER_QUANTITY", "MATERIAL_ID"]
for c in num_candidates:
    if c in df_clean.columns and df_clean[c].dtype == "object":
        df_clean[c] = to_num(df_clean[c])

# define target
if "made_a_purchase" not in df_clean.columns:
    raise ValueError("Expected 'made_a_purchase' column as target.")
df_clean["made_a_purchase"] = df_clean["made_a_purchase"].fillna(0).astype(int)

print("Cleaned frame shape:", df_clean.shape)
df_clean[["made_a_purchase"]].value_counts(normalize=True).rename("proportion")


In [ ]:
#Feature Engineering

#Transaction-based time features
if "transaction_date_df2" in df_clean.columns:
    df_clean["trans_dow"] = df_clean["transaction_date_df2"].dt.dayofweek
    df_clean["trans_hour"] = df_clean["transaction_date_df2"].dt.hour
    df_clean["trans_month"] = df_clean["transaction_date_df2"].dt.month
    df_clean["trans_is_weekend"] = df_clean["trans_dow"].isin([5, 6]).astype(int)

# Event-based date/time features
if "event_date_df1" in df_clean.columns:
    df_clean["event_dow"] = df_clean["event_date_df1"].dt.dayofweek
    df_clean["event_month"] = df_clean["event_date_df1"].dt.month
if "EVENT_TIMESTAMP" in df_clean.columns:
    df_clean["event_hour"] = df_clean["EVENT_TIMESTAMP"].dt.hour

# ITEMS feature
def items_count(val):
    if pd.isna(val):
        return np.nan
    try:
        parsed = ast.literal_eval(val)
        if isinstance(parsed, list):
            return len(parsed)
    except Exception:
        pass
    return np.nan

if "ITEMS" in df_clean.columns and df_clean["ITEMS"].dtype == "object":
    df_clean["items_count"] = df_clean["ITEMS"].apply(items_count)

# Device/channel features
if "DEVICE_CATEGORY" in df_clean.columns:
    df_clean["is_mobile"] = df_clean["DEVICE_CATEGORY"].isin(["mobile", "tablet"]).astype(int)
    df_clean["is_desktop"] = (df_clean["DEVICE_CATEGORY"] == "desktop").astype(int)

# Event action flags
if "EVENT_NAME" in df_clean.columns:
    df_clean["evt_add_to_cart"] = (df_clean["EVENT_NAME"] == "add_to_cart").astype(int)
    df_clean["evt_begin_checkout"] = df_clean["EVENT_NAME"].isin(
        ["begin_checkout", "proceed_to_checkout"]
    ).astype(int)
    df_clean["evt_purchase"] = (df_clean["EVENT_NAME"] == "purchase").astype(int)
    df_clean["evt_remove_from_cart"] = (df_clean["EVENT_NAME"] == "remove_from_cart").astype(int)

print("After feature engineering:", df_clean.shape)
display(df_clean.head())


In [ ]:
# Addressing Obvious Leakage

# Dropping columns that directly encode success / outcome / post-purchase info
possible_leakage_cols = [
    'purchase_successful',
    'EVENT_PAGE_NAME',
    'checkout_result',
    'order_status',
    'completed_flag',
    'transaction_id',
    'final_order_value',
    'payment_success',
    'is_converted',
    'order_complete_flag',
    'converted',
    'purchase_timestamp',
    'abandoned_in_window',
    'events_full_window',
    'actions_after_deadline',
    'evt_purchase',
]

df_clean = df_clean.drop(
    columns=[c for c in possible_leakage_cols if c in df_clean.columns],
    errors="ignore"
)

print("Columns after dropping obvious leakage:")
print(sorted(df_clean.columns))

# Removing explicit purchase rows from event-level data
work = df_clean.copy()

if "EVENT_NAME" in work.columns:
    work = work[work["EVENT_NAME"].ne("purchase")]

# Also rows whose event page looks like a success page
if "EVENT_PAGE_NAME" in work.columns:
    work = work[~work["EVENT_PAGE_NAME"].fillna("").str.contains("purchase success", case=False)]

# Making sure target stays intact
work["made_a_purchase"] = work["made_a_purchase"].fillna(0).astype(int)

print("Leak-filtered working frame shape:", work.shape)
display(work.head())


In [ ]:
#Modeling Frame
# categorical features
safe_categorical = [c for c in [
    "EVENT_NAME",
    "DEVICE_CATEGORY",
    "DEVICE_MOBILE_BRAND_NAME",
    "DEVICE_OPERATING_SYSTEM",
    "ORDER_TYPE",
] if c in work.columns]

# engineered numeric features
safe_numeric = [c for c in [
    "items_count",
    "trans_dow", "trans_hour", "trans_month", "trans_is_weekend",
    "event_dow", "event_hour", "event_month",
    "is_mobile", "is_desktop",
    "evt_add_to_cart", "evt_begin_checkout", "evt_remove_from_cart"
] if c in work.columns]

id_cols = [c for c in ["CUSTOMER_ID"] if c in work.columns]

cols_needed = (
    id_cols +
    (["event_date_df1"] if "event_date_df1" in work.columns else []) +
    safe_categorical + safe_numeric +
    ["made_a_purchase"]
)

df_leakfree = work[cols_needed].copy()

print("Leak-safe modeling frame shape:", df_leakfree.shape)
print("\nTarget distribution (made_a_purchase):")
print(df_leakfree["made_a_purchase"].value_counts(normalize=True).rename("proportion"))

display(df_leakfree.head())


In [ ]:
#Train/Test Split (Time-Based Holdout)

from sklearn.model_selection import GroupKFold, KFold
import pandas.api.types as ptypes

# Separate X and y
X = df_leakfree.drop(
    columns=["made_a_purchase", "event_date_df1"] if "event_date_df1" in df_leakfree.columns else ["made_a_purchase"]
)
y = df_leakfree["made_a_purchase"].copy()

# Group labels
groups = df_leakfree["CUSTOMER_ID"] if "CUSTOMER_ID" in df_leakfree.columns else None

# Time-based holdout: last 25% by event_date_df1; fallback to index order
if "event_date_df1" in df_leakfree.columns and ptypes.is_datetime64_any_dtype(df_leakfree["event_date_df1"]):
    order_idx = np.argsort(df_leakfree["event_date_df1"].values)
    cutoff = int(len(order_idx) * 0.75)
    train_idx, test_idx = order_idx[:cutoff], order_idx[cutoff:]
else:
    cutoff = int(len(X) * 0.75)
    train_idx, test_idx = np.arange(cutoff), np.arange(cutoff, len(X))

X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
groups_train = groups.iloc[train_idx] if groups is not None else None

print("Train/Test shapes:", X_train.shape, X_test.shape)
print("\nTarget distribution (train):")
print(y_train.value_counts(normalize=True).rename("proportion"))


# 6. Model Training & Evaluation

 Logistic regression model was trained using a preprocessed pipeline that handles:
- Imputation  
- One-hot encoding of categorical variables  
- Scaling numeric features  

The model was evaluated using:
- Cross-validated ROC-AUC  
- Time-based holdout ROC-AUC  
- Confusion matrix  
- Classification report  


In [ ]:
#Logistic Regression Pipeline + CV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix

# categorical vs numeric columns
cat_cols = [c for c in X_train.columns if X_train[c].dtype == "object"]
num_cols = [c for c in X_train.columns if X_train[c].dtype != "object"]

categorical_preprocess = Pipeline([
    ("imp", SimpleImputer(strategy="most_frequent")),
    ("oh", OneHotEncoder(handle_unknown="ignore"))
])

numeric_preprocess = Pipeline([
    ("imp", SimpleImputer(strategy="median")),
    ("sc", StandardScaler())
])

preproc = ColumnTransformer([
    ("cat", categorical_preprocess, cat_cols),
    ("num", numeric_preprocess, num_cols)
], remainder="drop")

logit = Pipeline([
    ("prep", preproc),
    ("model", LogisticRegression(max_iter=1000, class_weight="balanced", random_state=42))
])

# Cross-validation on TRAIN
if groups_train is not None:
    gkf = GroupKFold(n_splits=5)
    cv_scores = cross_val_score(
        logit, X_train, y_train,
        cv=gkf.split(X_train, y_train, groups_train),
        scoring="roc_auc"
    )
else:
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    cv_scores = cross_val_score(logit, X_train, y_train, cv=kf, scoring="roc_auc")

print("Logistic CV ROC-AUC:", np.round(cv_scores, 3), "Mean:", round(cv_scores.mean(), 3))

# Fit on TRAIN and evaluate on time-based TEST
logit.fit(X_train, y_train)
y_prob = logit.predict_proba(X_test)[:, 1]
y_pred = (y_prob >= 0.5).astype(int)

print("\nTime-based Test ROC-AUC:", round(roc_auc_score(y_test, y_prob), 3))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred, digits=3))


In [ ]:
#  ROC Curve, Brier Score & Calibration

import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve
from sklearn.calibration import calibration_curve
from sklearn.metrics import brier_score_loss

# ROC
fpr, tpr, thresholds = roc_curve(y_test, y_prob)
roc_auc = roc_auc_score(y_test, y_prob)
print("Time-based Test ROC-AUC:", round(roc_auc, 3))

plt.figure(figsize=(5,5))
plt.plot(fpr, tpr, label=f"Logistic (AUC={roc_auc:.3f})")
plt.plot([0,1], [0,1], "--", color="gray")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve – Purchase Prediction (Leak-safe)")
plt.grid(True)
plt.legend()
plt.show()

# Brier score
brier_model = brier_score_loss(y_test, y_prob)
base_prob = y_train.mean()
brier_baseline = brier_score_loss(y_test, np.full_like(y_test, fill_value=base_prob, dtype=float))

print("Brier score (model):", round(brier_model, 4))
print("Brier score (baseline, always = avg rate {:.3f}): {:.4f}".format(base_prob, brier_baseline))


# 7. Profit-Based Threshold Optimization

While classification metrics are useful, Our real objective is **recovering revenue **.  
Using assumptions for:
- Cost per contact  
- Expected profit per recovered purchase  

I computed net profit across thresholds from 0.01–0.99.  
The optimal threshold maximizes expected profit and provides a deployable decision rule.


In [ ]:
# Profit-Based Threshold Tuning
from sklearn.metrics import confusion_matrix

# Assumptions
cost_per_contact = 1.0
profit_per_purchase = 50.0
def profit_curve(y_true, y_prob, cost_per_contact, profit_per_purchase, n_thresholds=50):
    thresholds = np.linspace(0.01, 0.99, n_thresholds)
    rows = []
    n = len(y_true)
    for thr in thresholds:
        y_pred = (y_prob >= thr).astype(int)
        tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
        n_targeted = tp + fp

        revenue = tp * profit_per_purchase
        cost = n_targeted * cost_per_contact
        net_profit = revenue - cost
        profit_per_1000 = (net_profit / n) * 1000

        rows.append({
            "threshold": thr,
            "tp": tp,
            "fp": fp,
            "fn": fn,
            "tn": tn,
            "n_targeted": n_targeted,
            "revenue": revenue,
            "cost": cost,
            "net_profit": net_profit,
            "net_profit_per_1000": profit_per_1000
        })
    return pd.DataFrame(rows)

profit_df = profit_curve(y_test.values, y_prob, cost_per_contact, profit_per_purchase)
best_row = profit_df.loc[profit_df["net_profit"].idxmax()]

print("Best threshold by net profit:")
display(best_row)

plt.figure(figsize=(6,4))
plt.plot(profit_df["threshold"], profit_df["net_profit_per_1000"])
plt.axvline(best_row["threshold"], linestyle="--", label=f"Best thr={best_row['threshold']:.2f}")
plt.title("Net Profit per 1,000 Customers vs Threshold")
plt.xlabel("Decision Threshold (p̂ ≥ threshold → contact)")
plt.ylabel("Net Profit per 1,000 Customers")
plt.grid(True)
plt.legend()
plt.show()


# 9. Final Recommendations & Next Steps

- Target customers above the optimal probability threshold for outreach, maximizing revenue recovery.  
- Focus user-experience improvements on the most common drop-off points identified in GA events.    
- Consider A/B testing different outreach strategies for customers at varying risk levels.  
